In [ ]:
###Decision Tree####
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from IPython.display import Image  
import pydotplus

clfD = DecisionTreeClassifier(criterion="gini",random_state=0, max_depth=5)
cv = StratifiedKFold(n_splits=2, random_state=0, shuffle=True)

totScore=0.0
i=0

#print(dfSelectedX.head())
for train_index,test_index in cv.split(dfSelectedX,dfSelectedY):
    x_train, x_test = dfSelectedX.iloc[train_index], dfSelectedX.iloc[test_index]
    y_train, y_test = dfSelectedY.iloc[train_index], dfSelectedY.iloc[test_index]
    #clfD.fit(dfSelectedX, dfSelectedY.values.ravel())
    clfD.fit(x_train, y_train.values.ravel())
    predictions = clfD.predict(x_test)

    predY = pd.DataFrame(predictions,index= y_test.index, columns=['Predicted_type'])
    #print(predY)

    finalPred = pd.concat([y_test,predY],axis=1)
    #print(finalPred)
    #print(multilabel_confusion_matrix(y_test, predictions))
    print("\n-----------------------------")
    tn,fp,fn,tp = confusion_matrix(y_test, predictions).ravel()
    print(" tn\tfp\tfn\t tp\n",tn,"\t",fp,"\t",fn,"\t",tp)
    score = clfD.score(x_test, y_test)
    print("\naccuracy:",score)
    
    print("-----------------------------\n")
    score = clfD.score(x_test, y_test)
    #print(score)
    
    totScore = totScore + score
    i=i+1
    
print("average", totScore/i)



In [ ]:
from sklearn.tree import export_graphviz
# Export as dot file
dot_data = export_graphviz(clfD, 
                out_file=None, 
                feature_names=dfSelectedX.columns,
                class_names=['Alive','Dead'],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

graph = pydotplus.graph_from_dot_data(dot_data)  

# Show graph
Image(graph.create_png())

In [ ]:
####SVM####
from sklearn import svm
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

clf = svm.SVC(kernel='rbf',gamma='auto',decision_function_shape='ovr')
cv = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)

scaler = StandardScaler()

totScore=0
totSensitivity = 0
totSpecificity = 0
totAUC = 0
i=0

for train_index,test_index in cv.split(dfSelectedX,dfSelectedY):
    x_train, x_test = dfSelectedX.iloc[train_index], dfSelectedX.iloc[test_index]
    y_train, y_test = dfSelectedY.iloc[train_index], dfSelectedY.iloc[test_index]
    #clf.fit(dfSelectedX, dfSelectedY.values.ravel())
    clf.fit(x_train, y_train.values.ravel())
    predictions = clf.predict(x_test)

    predY = pd.DataFrame(predictions,index= y_test.index, columns=['Predicted_type'])
    #print(predY)

    finalPred = pd.concat([y_test,predY],axis=1)
    #print(finalPred)
    #print(multilabel_confusion_matrix(y_test, predictions))
    #print("\n-----------------------------")
    tn,fp,fn,tp = confusion_matrix(y_test, predictions).ravel()
    #print(" tn\tfp\tfn\t tp\n",tn,"\t",fp,"\t",fn,"\t",tp)
    score = clf.score(x_test, y_test)
    sensitivity = tp*100/(tp+fn)
    specificity = tn*100/(tn+fp)
    fpr, tpr, thresholds = metrics.roc_curve(y_test, predictions)
    auc = metrics.auc(fpr, tpr)
    #print("\naccuracy:",score)
    
    #print("-----------------------------\n")
    
    totScore = totScore + score
    totSensitivity = totSensitivity + sensitivity
    totSpecificity = totSpecificity + specificity
    totAUC = totAUC + auc
    i=i+1
    
print("average accuracy:", totScore/i)
print("average sensitivity:", totSensitivity/i)
print("average specificity:", totSpecificity/i)
print("average AUC:", totAUC/i)